In [28]:
import json
import base64
import qrcode
import requests
import datetime
import uuid
import matplotlib.pyplot as plt
from pymongo import MongoClient
from typing import Optional, List
from didcomm.common.types import DID, VerificationMethodType, VerificationMaterial, VerificationMaterialFormat
from didcomm.did_doc.did_doc import DIDDoc, VerificationMethod, DIDCommService
from didcomm.did_doc.did_resolver import DIDResolver
from didcomm.message import Message, FromPrior
from didcomm.secrets.secrets_resolver_demo import SecretsResolverDemo
from didcomm.unpack import unpack, UnpackResult
from didcomm.common.resolvers import ResolversConfig
from didcomm.pack_encrypted import pack_encrypted, PackEncryptedConfig, PackEncryptedResult
from peerdid.core.did_doc_types import DIDCommServicePeerDID
from didcomm.secrets.secrets_util import generate_x25519_keys_as_jwk_dict, generate_ed25519_keys_as_jwk_dict, jwk_to_secret
from peerdid import peer_did
from peerdid.did_doc import DIDDocPeerDID
from peerdid.types import VerificationMaterialAuthentication, VerificationMethodTypeAuthentication, VerificationMaterialAgreement, VerificationMethodTypeAgreement, VerificationMaterialFormatPeerDID

In [29]:
secrets_resolver = SecretsResolverDemo()

In [30]:
class DIDResolverPeerDID(DIDResolver):
    async def resolve(self, did: DID) -> DIDDoc:
        did_doc_json = peer_did.resolve_peer_did(did, format = VerificationMaterialFormatPeerDID.JWK)
        did_doc = DIDDocPeerDID.from_json(did_doc_json)

        return DIDDoc(
            did=did_doc.did,
            key_agreement_kids = did_doc.agreement_kids,
            authentication_kids = did_doc.auth_kids,
            verification_methods = [
                VerificationMethod(
                    id = m.id,
                    type = VerificationMethodType.JSON_WEB_KEY_2020,
                    controller = m.controller,
                    verification_material = VerificationMaterial(
                        format = VerificationMaterialFormat.JWK,
                        value = json.dumps(m.ver_material.value)
                    )
                )
                for m in did_doc.authentication + did_doc.key_agreement
            ],
            didcomm_services = [
                DIDCommService(
                    id = s.id,
                    service_endpoint = s.service_endpoint,
                    routing_keys = s.routing_keys,
                    accept = s.accept
                )
                for s in did_doc.service
                if isinstance(s, DIDCommServicePeerDID)
            ] if did_doc.service else []
        )

In [31]:
async def create_peer_did(self,
                        auth_keys_count: int = 1,
                        agreement_keys_count: int = 1,
                        service_endpoint: Optional[str] = None,
                        service_routing_keys: Optional[List[str]] = None
                        ) -> str:
        # 1. generate keys in JWK format
        agreem_keys = [generate_x25519_keys_as_jwk_dict() for _ in range(agreement_keys_count)]
        auth_keys = [generate_ed25519_keys_as_jwk_dict() for _ in range(auth_keys_count)]

        # 2. prepare the keys for peer DID lib
        agreem_keys_peer_did = [
            VerificationMaterialAgreement(
                type=VerificationMethodTypeAgreement.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in agreem_keys
        ]
        auth_keys_peer_did = [
            VerificationMaterialAuthentication(
                type=VerificationMethodTypeAuthentication.JSON_WEB_KEY_2020,
                format=VerificationMaterialFormatPeerDID.JWK,
                value=k[1],
            )
            for k in auth_keys
        ]

        # 3. generate service
        service = None
        if service_endpoint:
            service = json.dumps(
                DIDCommServicePeerDID(
                    id="new-id",
                    service_endpoint=service_endpoint, routing_keys=service_routing_keys,
                    accept=["didcomm/v2"]
                ).to_dict()
            )

        # 4. call peer DID lib
        # if we have just one key (auth), then use numalg0 algorithm
        # otherwise use numalg2 algorithm
        if len(auth_keys_peer_did) == 1 and not agreem_keys_peer_did and not service:
            did = peer_did.create_peer_did_numalgo_0(auth_keys_peer_did[0])
        else:
            did = peer_did.create_peer_did_numalgo_2(
                encryption_keys=agreem_keys_peer_did,
                signing_keys=auth_keys_peer_did,
                service=service,
            )

        # 5. set KIDs as in DID DOC for secrets and store the secret in the secrets resolver
        did_doc = DIDDocPeerDID.from_json(peer_did.resolve_peer_did(did))
        for auth_key, kid in zip(auth_keys, did_doc.auth_kids):
            private_key = auth_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        for agreem_key, kid in zip(agreem_keys, did_doc.agreement_kids):
            private_key = agreem_key[0]
            private_key["kid"] = kid
            print(private_key)
            await secrets_resolver.add_key(jwk_to_secret(private_key))

        return did


### Alice get OOB from Mediator

In [32]:
#oob_url = requests.get("http://127.0.0.1:8000/oob_url").text
oob_url = requests.get("https://mediator.rootsid.cloud/oob_url").text

In [33]:
received_msg_encoded = oob_url.split("=")[1]
received_msg_decoded = json.loads(str(base64.urlsafe_b64decode(received_msg_encoded + "=="), "utf-8"))
alice_did_for_mediator = await create_peer_did(1,1)
print("Alice's DID:", alice_did_for_mediator)

{'crv': 'Ed25519', 'x': '5XByq3PnKm4g6Zg0QXYDG-dLENT6NsW2lrIZ1-OYM5c', 'd': 'pUelY8s3kLqkqMU7w-WwDt9_3IakHwerHYkkzadW6O8', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LStYZzARSwPUHs373XjneswGFXDBwL9fC9DAwHaMx6oH3i.Vz6Mkuttrhqo6dykxbqVQ84xvWQaxsoMQva1k633fXuWJGdXt#6Mkuttrhqo6dykxbqVQ84xvWQaxsoMQva1k633fXuWJGdXt'}
{'crv': 'X25519', 'x': '-qPV4qHVn7EdZf-T9oXMCAdhsRmuTOHpSy_VUvx39D0', 'd': '2KnEJFoqGkLomGdIep2PWflFisDm2Uw-qPfH-v46lF8', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LStYZzARSwPUHs373XjneswGFXDBwL9fC9DAwHaMx6oH3i.Vz6Mkuttrhqo6dykxbqVQ84xvWQaxsoMQva1k633fXuWJGdXt#6LStYZzARSwPUHs373XjneswGFXDBwL9fC9DAwHaMx6oH3i'}
Alice's DID: did:peer:2.Ez6LStYZzARSwPUHs373XjneswGFXDBwL9fC9DAwHaMx6oH3i.Vz6Mkuttrhqo6dykxbqVQ84xvWQaxsoMQva1k633fXuWJGdXt


### Alice request mediate

In [34]:
alice_mediate_grant = Message(
    custom_headers = [{"return_route": "all"}],
    id = str(uuid.uuid4()),
    type = "https://didcomm.org/coordinate-mediation/2.0/mediate-request",
    body = {}
)

In [35]:
alice_mediate_grant_packed = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_mediate_grant,
    frm = alice_did_for_mediator,
    to = received_msg_decoded["from"],
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

### Sending to Mediator

In [36]:
mediator_did_doc = json.loads(peer_did.resolve_peer_did(received_msg_decoded["from"]))
mediator_endpoint = mediator_did_doc["service"][0]["serviceEndpoint"]
print(mediator_endpoint)

https://mediator.rootsid.cloud


In [37]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_mediate_grant_packed.packed_msg)


In [38]:
print(resp.json())

{'message': 'Service Unavailable'}


### Get mediator grant response

In [19]:
mediator_unpack_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg.message.type)
print("Routing Key: " + mediator_unpack_msg.message.body["routing_did"])
print("Mediator new DID: " + mediator_unpack_msg.message.from_prior.sub)
# mediator rotated did
mediator_routing_key = mediator_unpack_msg.message.body["routing_did"]
mediator_did = mediator_unpack_msg.message.from_prior.sub

MalformedMessageError: MalformedMessageCode.INVALID_PLAINTEXT

### Keylist update

Alice create a new DID to share with  other peers using mediator routing keys 

In [56]:
#alice_did_new = await create_peer_did(1, 1, service_endpoint=mediator_endpoint, service_routing_keys=[mediator_routing_key])
alice_did_for_bob = await create_peer_did(1, 1, service_endpoint=[{"uri": mediator_routing_key}])

print("Alice's DID for Bob:", alice_did_for_bob)

{'crv': 'Ed25519', 'x': '2b2ePwb2rEl3q9FjbLMoF4ZcKAs4eJUSHta_TTxgdQM', 'd': 'D7hojt0265TRdORsE38UEd5ww3jyuEfZ4YrhAJrT51w', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSoqh2gx4zZyKwrsKZMaHmzdnsrfZkhozrNgnKYbF5Fhji.Vz6Mku7EDS1Eujjxr3cxEykJSxjTgVSVB6cXm9grSsosbD1nJ.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNvbTlKMThCeFBCSFlIWmJ3cnpBZXVNeDV5cTR2dDdSZGVpVEdZZWFBaTFjZC5WejZNa29NbnpzbTlXamVwYkJiTFNmVHdpQlZiUmkyUUtGRHpxVDdya1FiaXNua29ULlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9XSwiYSI6WyJkaWRjb21tL3YyIl19#6Mku7EDS1Eujjxr3cxEykJSxjTgVSVB6cXm9grSsosbD1nJ'}
{'crv': 'X25519', 'x': 'tLw9ITSsMb3iqnWceAi37D8jSseBd4mLfto6d9y3Wn0', 'd': 'cOQMUnKI9JnqcyooHaUPMeRFr40IL_YRSPnDxSkpjVA', 'kty': 'OKP', 'kid': 'did:peer:2.Ez6LSoqh2gx4zZyKwrsKZMaHmzdnsrfZkhozrNgnKYbF5Fhji.Vz6Mku7EDS1Eujjxr3cxEykJSxjTgVSVB6cXm9grSsosbD1nJ.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNvbTlKMThC

Alice create and send the keylist update message

In [57]:
alice_keylist_update = Message(
    id = "unique-id-293e9a922efff",
    type = "https://didcomm.org/coordinate-mediation/2.0/keylist-update",
    body = {
        "updates":[
        {
            "recipient_did": alice_did_for_bob,
            "action": "add"
        }
    ]
    }
)

In [58]:
alice_keylist_update_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_keylist_update,
    frm = alice_did_for_mediator,
    to = mediator_did,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)

In [59]:
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp = requests.post(mediator_endpoint, headers=headers, data = alice_keylist_update_packed_msg.packed_msg)

In [60]:
mediator_unpack_msg2 = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp.json()
)
print(mediator_unpack_msg2.message.type)
print(mediator_unpack_msg2.message.body)


https://didcomm.org/coordinate-mediation/2.0/keylist-update-response
{'updated': [{'recipient_did': 'did:peer:2.Ez6LSoqh2gx4zZyKwrsKZMaHmzdnsrfZkhozrNgnKYbF5Fhji.Vz6Mku7EDS1Eujjxr3cxEykJSxjTgVSVB6cXm9grSsosbD1nJ.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6ImRpZDpwZWVyOjIuRXo2TFNvbTlKMThCeFBCSFlIWmJ3cnpBZXVNeDV5cTR2dDdSZGVpVEdZZWFBaTFjZC5WejZNa29NbnpzbTlXamVwYkJiTFNmVHdpQlZiUmkyUUtGRHpxVDdya1FiaXNua29ULlNleUpwWkNJNkltNWxkeTFwWkNJc0luUWlPaUprYlNJc0luTWlPaUpvZEhSd09pOHZNVEkzTGpBdU1DNHhPamd3TURBaUxDSmhJanBiSW1ScFpHTnZiVzB2ZGpJaVhYMCJ9XSwiYSI6WyJkaWRjb21tL3YyIl19', 'action': 'add', 'result': 'success'}]}


### Alice can now send her DID to Bob.

### Alice check status of queued messages from the Mediator

In [61]:
alice_status_check = Message(
    id = "unique-id-293e9a922efffxxx",
    type = "https://didcomm.org/messagepickup/3.0/status-request",
    body = {}
)

In [111]:
alice_status_check_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_status_check,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp3 = requests.post(mediator_endpoint, headers=headers, data = alice_status_check_packed_msg.packed_msg)

In [112]:
mediator_unpack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp3.json()
)
print(mediator_unpack_status.message.type)
print("Messages in Mediator queue: " + str(mediator_unpack_status.message.body["message_count"]))

https://didcomm.org/messagepickup/3.0/status
Messages in Mediator queue: 5


### If there are messages, Alice can pickup from the mediator (delivery request)

In [124]:
alice_delivery_request = Message(
    id = "unique-id-293e9a922efffxxxff",
    type = "https://didcomm.org/messagepickup/3.0/delivery-request",
    body = {"limit": 1}
)

In [125]:
alice_delivery_request_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_delivery_request,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp4 = requests.post(mediator_endpoint, headers=headers, data = alice_delivery_request_packed_msg.packed_msg)

In [126]:
mediator_delivery = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp4.json()
)
print(mediator_delivery.message.type)
print(mediator_delivery.message.body)

https://didcomm.org/messagepickup/3.0/delivery
{'recipient_key': None}


In [127]:
bob_packed_msg = mediator_delivery.message.attachments[0].data.json
msg_id = mediator_delivery.message.attachments[0].id

In [128]:
print(bob_packed_msg)

{'protected': 'eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC0xUFUrQTI1NktXIiwiZW5jIjoiQTI1NkNCQy1IUzUxMiIsImFwdSI6IlpHbGtPbkJsWlhJNk1pNUZlalpNVTNSeFUxRlVlVEZ1U25rNFZrVldURkZ6YVdaMU1URjVZMlZoVFhWeGNWZHRWSFZMZUdseWVHWTNkR1ZLTGxaNk5rMXJabnB6V1d0dGJsaEtXSGRCYTFRMlRGSmFVRlpFUVhkNWFURkdaRlpqV0RReVpuQTNNVU4xY1hRNGNtUXVVMlY1U25CYVEwazJTVzAxYkdSNU1YQmFRMGx6U1c1UmFVOXBTbXRpVTBselNXNU5hVTlwU205a1NGSjNZM3B2ZGt3elpETmtlVFZzWlVkR2RHTkhlR3hNYlU1MllsTTVhV0l5U1dsTVEwcG9TV3B3WWtsdFVuQmFSMDUyWWxjd2RtUnFTV2xZV0RBak5reFRkSEZUVVZSNU1XNUtlVGhXUlZaTVVYTnBablV4TVhsalpXRk5kWEZ4VjIxVWRVdDRhWEo0WmpkMFpVbyIsImFwdiI6Ik44N21JX0dWUHMtNm54LWVPb09UZktUX3JqWjRLbmZmaVMyWVVVXzVoZkEiLCJza2lkIjoiZGlkOnBlZXI6Mi5FejZMU3RxU1FUeTFuSnk4VkVWTFFzaWZ1MTF5Y2VhTXVxcVdtVHVLeGlyeGY3dGVKLlZ6Nk1rZnpzWWttblhKWHdBa1Q2TFJaUFZEQXd5aTFGZFZjWDQyZnA3MUN1cXQ4cmQuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3Y3pvdkwzZDNkeTVsZUdGdGNHeGxMbU52YlM5aWIySWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAjNkxTdHFTUVR5MW5Ke

In [129]:
bob_msg = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= bob_packed_msg
)

In [130]:
print("Message ID:", msg_id)
print(bob_msg.message.body["content"])

Message ID: 62fa69fc76d05e5e6b96c65b
Hola Alice


### Alice acknowledge so the mediator can delete the message

In [131]:
alice_ack = Message(
    id = "unique-id-293e9a922efffxxxffsss",
    type = "https://didcomm.org/messagepickup/3.0/messages-received",
    body = {"message_id_list": [msg_id]}
)

In [132]:
alice_ack_packed_msg = await pack_encrypted(
    resolvers_config = ResolversConfig(
        secrets_resolver = secrets_resolver,
        did_resolver = DIDResolverPeerDID()
    ),
    message = alice_ack,
    to = mediator_did,
    frm = alice_did_for_mediator,
    sign_frm = None,
    pack_config = PackEncryptedConfig(protect_sender_id=False)
)
headers = {"Content-Type": "application/didcomm-encrypted+json"}
resp5 = requests.post(mediator_endpoint, headers=headers, data = alice_ack_packed_msg.packed_msg)

In [133]:
mediator_ack_status = await unpack(
    resolvers_config=ResolversConfig(
        secrets_resolver=secrets_resolver,
        did_resolver=DIDResolverPeerDID()
    ),
    packed_msg= resp5.json()
)
print(mediator_ack_status.message.type)
print(mediator_ack_status.message.body)

https://didcomm.org/messagepickup/3.0/status
{'message_count': 3, 'live_delivery': False}


-------------------

In [44]:
mongo = MongoClient()

In [54]:
db = mongo.mediator


In [75]:
con = db.connections.find_one({"remote_did":alice_did})

In [76]:
print(con["keylist"])

['did:peer:2.Ez6LSq3s3WPhCnY26D6zSs4YUKoLMFpWwdQHWt7AMRUtTpx9U.Vz6MkuNqx9roenhEFxPe9oQrZH2re6iHmgdE8cEJ1kB9W422d.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOlt7InVyaSI6Imh0dHA6Ly8xMjcuMC4wLjE6ODAwMCIsImEiOlsiZGlkY29tbS92MiIsImRpZGNvbW0vYWlwMjtlbnY9cmZjNTg3Il0sInIiOlsiZGlkOnBlZXI6Mi5FejZMU2luN002bUt5aXUxd3liU0dUZFo5NVBhSFl6N0RERmk0WDFqQWc5UDN3a2VnLlZ6Nk1ra1ZBcGdnbWlxRGU4SkR3MTRpZVE2UTR3UW5yUUVxZVA4ZVVHdkxRendkTHcuU2V5SnBaQ0k2SW01bGR5MXBaQ0lzSW5RaU9pSmtiU0lzSW5NaU9pSm9kSFJ3T2k4dk1USTNMakF1TUM0eE9qZ3dNREFpTENKaElqcGJJbVJwWkdOdmJXMHZkaklpWFgwIl19XSwiYSI6WyJkaWRjb21tL3YyIl19']


In [63]:
msgs = db.messages.find({"recipient_key": {"$in":con["keylist"]}},{"attachment":1}).limit(10)

In [111]:
rec_key = "did:peer:2.Ez6LSi2yQqo79XoGN2dVFHLvS7y4wgnUP5MnvDLoeCenWCVMj.Vz6MkjUDxPJFqo1PCxsTjU5W6N1R9S2Bd2MiW85BuwJ21RWYP.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6WyJkaWQ6cGVlcjoyLkV6NkxTb1Q5ajZMN2oyV3lHeXhqNk5XREZablVrRno1Mkg1ZjVNNzV3clB2MVA4R3AuVno2TWtpVGltRk45dXNhMmtiMUs2cmRRQkRSY1dSVUw4akNUbVJTRXF1SmtFdkYxQS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndPaTh2TVRJM0xqQXVNQzR4T2pnd01EQWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiXX0"

In [125]:
msgs = db.messages.find({"recipient_key": {"$in":[rec_key]}},{"attachment":1}).limit(10)

In [126]:
mm= list(msgs)

In [128]:
len(mm)

1

In [276]:
list(db.messages.find({"_id": "62d8765c706ccae77c78d493"}))

[{'_id': ObjectId('62d7fad0c5799100ef16655a'),
  'recipient_key': 'did:peer:2.Ez6LSi2yQqo79XoGN2dVFHLvS7y4wgnUP5MnvDLoeCenWCVMj.Vz6MkjUDxPJFqo1PCxsTjU5W6N1R9S2Bd2MiW85BuwJ21RWYP.SeyJpZCI6Im5ldy1pZCIsInQiOiJkbSIsInMiOiJodHRwOi8vMTI3LjAuMC4xOjgwMDAiLCJhIjpbImRpZGNvbW0vdjIiXSwiciI6WyJkaWQ6cGVlcjoyLkV6NkxTb1Q5ajZMN2oyV3lHeXhqNk5XREZablVrRno1Mkg1ZjVNNzV3clB2MVA4R3AuVno2TWtpVGltRk45dXNhMmtiMUs2cmRRQkRSY1dSVUw4akNUbVJTRXF1SmtFdkYxQS5TZXlKcFpDSTZJbTVsZHkxcFpDSXNJblFpT2lKa2JTSXNJbk1pT2lKb2RIUndPaTh2TVRJM0xqQXVNQzR4T2pnd01EQWlMQ0poSWpwYkltUnBaR052YlcwdmRqSWlYWDAiXX0',
  'attachment': '{"protected":"eyJ0eXAiOiJhcHBsaWNhdGlvbi9kaWRjb21tLWVuY3J5cHRlZCtqc29uIiwiYWxnIjoiRUNESC1FUytBMjU2S1ciLCJlbmMiOiJYQzIwUCIsImFwdiI6InJVNjNTbHAyc1JUc1hMUXd6S195M3VReXBDeV9WQW0taVpYVjRFUVBXemsiLCJlcGsiOnsiY3J2IjoiWDI1NTE5IiwieCI6IkZSX3FOOXFpRTRubnk5X1JDWFd4VWEyRzdmYVN4NGZUb1VTTE9mLVNnQXciLCJrdHkiOiJPS1AifX0","recipients":[{"header":{"kid":"did:peer:2.Ez6LSoT9j6L7j2WyGyxj6NWDFZnUkFz52H5f5M75wrPv1P8Gp.Vz6MkiTimFN9usa2kb

In [299]:
list(db.oobs.find())

[]

In [290]:
db.oobs.delete_one({"date": 1657500891000})